In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Mataura,NZ,-46.19,168.86,39.00,83,74,3.00
1,1,Vardo,NO,70.37,31.11,48.20,81,75,16.11
2,2,Vanderhoof,CA,54.02,-124.02,56.23,55,83,12.10
3,3,Ancud,CL,-41.87,-73.82,50.00,75,75,6.93
4,4,Rikitea,PF,-23.12,-134.97,71.06,71,0,17.67


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [6]:
#1 Assign the locations to an array of latitude and longtiude pairs
locations = city_data_df[["Lat", "Lng"]]

#2 Assign the weights variable to some variables
max_temp = city_data_df["Max Temp"]

#3 Assign the figure variable gmaps.figure() attribute
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# 4. Assign the heatmap layer to the heatmap attribute and add in the locations
heatmap_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)

#5 Add the heatmap layer
fig.add_layer(heatmap_layer)
#6 Call the figure to plot the data
fig 

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of Wind Speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
9,9,Carutapera,BR,-1.20,-46.02,87.73,55,43,4.94
11,11,Mhango,TZ,-3.28,32.85,76.19,37,59,4.29
19,19,Yumen,CN,40.28,97.20,76.12,25,83,4.94
22,22,Atuona,PF,-9.80,-139.03,79.93,76,29,24.29
24,24,Dingle,PH,11.00,122.67,77.76,87,91,2.26


In [12]:
preferred_cities_df.count()

City_ID       213
City          213
Country       213
Lat           213
Lng           213
Max Temp      213
Humidity      213
Cloudiness    213
Wind Speed    213
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Carutapera,BR,87.73,-1.20,-46.02,
11,Mhango,TZ,76.19,-3.28,32.85,
19,Yumen,CN,76.12,40.28,97.20,
22,Atuona,PF,79.93,-9.80,-139.03,
24,Dingle,PH,77.76,11.00,122.67,
25,Aleksandrov Gay,RU,80.04,50.15,48.57,
29,Nouadhibou,MR,82.40,20.93,-17.03,
38,Vostochnyy,RU,79.00,55.82,37.87,
39,Teguise,ES,80.60,29.06,-13.56,
41,Kapaa,US,77.00,22.08,-159.32,


In [14]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [28]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
# Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... Skipping")

In [33]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
9,Carutapera,BR,87.73,-1.20,-46.02,
11,Mhango,TZ,76.19,-3.28,32.85,
19,Yumen,CN,76.12,40.28,97.20,
22,Atuona,PF,79.93,-9.80,-139.03,
24,Dingle,PH,77.76,11.00,122.67,
25,Aleksandrov Gay,RU,80.04,50.15,48.57,
29,Nouadhibou,MR,82.40,20.93,-17.03,
38,Vostochnyy,RU,79.00,55.82,37.87,
39,Teguise,ES,80.60,29.06,-13.56,
41,Kapaa,US,77.00,22.08,-159.32,


In [37]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [39]:
 #Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))